## Assigmnent 3

In [1]:
from PattRecClasses import DiscreteD, GaussD, HMM, MarkovChain
from matplotlib import pyplot as plt

import os
import numpy as np

### Feature extraction

In [2]:

acc = []

for filename in os.listdir('sensorlogger'):
    path = os.path.join('sensorlogger',filename)
    with open(path) as file:        
        if os.path.isfile(path):
            lines = [line.split() for line in file]
            # absolute value of norm of acceleration vector - gravity
            acc.append([ np.abs((np.sqrt(float(line[2])**2 + float(line[3])**2 + float(line[4])**2))-9.82) for line in lines ])

%matplotlib qt
plt.figure(figsize= (10,5))
plt.plot(acc[0])
plt.title("Magnitude of l2-norm of Acceleration (Gravity Subtracted)")
plt.xlabel("Sample")
plt.ylabel("Amplitude")

plt.show()


/nix/store/b2hc0i92l22ir2kavnjn3z5z6mzabbvm-glibc-2.34-210/lib/libc.so.6: version `GLIBC_2.38' not found (required by /nix/store/6j6hl9x71gpq4siar7ns8w2fwx0s7d6q-gvfs-1.52.2/lib/gio/modules/libgvfsdbus.so)
Failed to load module: /nix/store/6j6hl9x71gpq4siar7ns8w2fwx0s7d6q-gvfs-1.52.2/lib/gio/modules/libgvfsdbus.so
QApplication: invalid style override 'gtk2' passed, ignoring it.
	Available styles: bb10dark, bb10bright, cleanlooks, gtk2, cde, motif, plastique, Windows, Fusion
qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBuff

In [3]:

"""Windowing"""
wLen = 100 # window length (samples)
overlap = 0 # is 
windows = []

step = wLen-overlap

X = []

count = 0
for rec in acc:
    x = []
    for n in range(0,len(rec)-wLen +1,step):
        x.append( sum(rec[n:n+wLen])/wLen )
        
  #  print(np.shape(x))
    
    """divide recordings into chunks of 50 windows"""
    #####
    nchunks = len(x)//50
    chunks = [x[n:n+50] for n in range(0, len(x), 50)]    
    if len(chunks[-1]) <40 and len(x)>= 50:
        chunks[-2].extend(chunks[-1])
        chunks = chunks[:-1]        
    X.extend(chunks)
    count = count + nchunks+1 
    #####
    
plt.figure(figsize= (10,5))
plt.plot(X[0])
plt.title("Means of Segments of Acceleration")
plt.xlabel("Window")
plt.ylabel("Amplitude")

plt.show()


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


In [4]:
"""Cluster training data into N separate categories, for estimation of output probabilities"""
data1 = [] 
data2 = []
data3 = []

"""Automatic labeling"""
S = [] # labels
for rec in X:
    s = []
    for a in rec:
        if a > 3.5:
            s.append(3) #running
            data3.append(a)
        elif a > 0.6:
            s.append(2)  #walking
            data2.append(a)
        else:
            s.append(1)  #standing
            data1.append(a)
    S.append(s)  
        
colors = {1: 'red', 2: 'green', 3: 'blue'}

plt.figure(figsize= (10,5))
plt.plot([t2 for t2 in range(0, len(X[0]))], X[0])
for i, val in enumerate(S[0]):
    plt.scatter(i, val, c = colors[val])
for key in colors: 
    plt.scatter([],[], c=colors[key], label = f'S = {key}')
    
plt.legend()
    
plt.legend()
plt.title("Labeled Segments")
plt.xlabel("Window")
plt.ylabel("Amplitude")

plt.show()

qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


In [5]:
"""All plots"""

plt.figure(figsize= (15,10))

plt.subplot(311)
plt.plot(acc[0])

plt.subplot(312)
plt.plot(X[0])

plt.subplot(313)
plt.plot([t2 for t2 in range(0, len(X[0]))], X[0])
for i, val in enumerate(S[0]):
    plt.scatter(i, val, color = colors[val])

plt.show()



qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


In [6]:

fig, (ax1,ax2,ax3) = plt.subplots(3)

ax1.hist(data1, bins = 30)
ax2.hist(data2, bins = 30)
ax3.hist(data3, bins = 30)
plt.show()


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.


In [7]:
plt.hist(data2, bins = 30)
plt.title("Feature Sample Amplitudes")
plt.xlabel("Amplitude")
plt.ylabel("Count")
plt.show()

## Train/Test Split

In [8]:

from sklearn.model_selection import train_test_split

datapoints = np.array([X,S], dtype = object).T

X_train, X_test, y_train, y_test = train_test_split( X,S, test_size = 0.33, random_state = 1)


In [9]:
"""Exporting data to csv"""

import csv

with open('features.csv', 'w') as csvfile:
    Writer = csv.writer(csvfile)
    for data in datapoints:
        Writer.writerow(data)
        

In [10]:
"""Initialization"""

"""Infinite ergodic HMM may start in any state. Assumption here is that
    each state is equally likely to occur as initial state"""

mc = MarkovChain( np.full( (1,3),1/3 ) , np.array( [ [0.9, 0.09, 0.01],
                                                     [0.05, 0.9, 0.05],
                                                     [0.01, 0.09, 0.9] ] ))

"""The diagonal elements of the transition matrix can be chosen to indicate 
    approximately the average state durations. Here I assume that each state duration is equally long"""

from scipy.stats import norm
        
mu1, std1 = norm.fit(data1) #standing
mu2, std2 = norm.fit(data2) #walking
mu3, std3 = norm.fit(data3) #running

# Looks reasonable
print(mu1,std1)
print(mu2,std2)
print(mu3,std3)


b1 = GaussD( means=[mu1], stdevs=[std1] )   # Distribution for state = standing
b2 = GaussD( means=[mu2], stdevs=[std2] )   # Distribution for state = walking
b3 = GaussD( means=[mu3], stdevs=[std3] )   # Distribution for state = running

"""Initial HMM"""

h  = HMM( mc, [b1, b2, b3])

0.2162458875181304 0.12492265518154028
1.7585476077498798 0.44732077037465684
6.011873669473138 0.990593615394949


### Training

In [11]:
h.train(X_train)

print(h.stateGen.q)
print(h.stateGen.A)

[[0.16756731 0.55660027 0.27583242]]
[[0.85966746 0.13281941 0.00751313]
 [0.03885805 0.92395303 0.03718892]
 [0.00379829 0.06095733 0.93524439]]


## Evaluation

In [20]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

test_seq = []
for x_obs in X_test:
    test_seq.extend(h.viterbi(x_obs))
    
y_true = []
for y in y_test:
    y_true.extend([j for j in y])

count = 0
for i, j in zip(y_true,test_seq):
    if i == j:
        count = count+1
        
print(f"Correct classifications: {count/len(y_true)} %")

cm = confusion_matrix(y_true, test_seq)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = ('Standing', 'Walking', 'Running')) 

disp.plot()
plt.show()


Correct classifications: 0.9959183673469387 %


qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
Falling back to using screens root_visual.
